In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# تحميل البيانات
df_mat = pd.read_csv("student-mat.csv", sep=';')
df_por = pd.read_csv("student-por.csv", sep=';')
df = pd.concat([df_mat, df_por], ignore_index=True)

# ترميز النصوص
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])

# إنشاء target
df['target'] = (df['G3'] >= 10).astype(int)
df.drop(columns=['G3'], inplace=True)

# تحضير البيانات
X = df.drop(columns=['target'])
y = df['target']
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# تعريف النماذج والمعلمات
models_params = {
    "SVM": {
        "model": SVC(),
        "params": {
            "kernel": ["linear", "rbf"],
            "C": [0.1, 1, 10, 100],
            "gamma": [0.001, 0.01, 0.1, 1]
        }
    },
    "Decision Tree": {
        "model": DecisionTreeClassifier(),
        "params": {
            "max_depth": [3, 5, 7, 10],
            "min_samples_split": [2, 5, 10, 20]
        }
    },
    "Naive Bayes": {
        "model": GaussianNB(),
        "params": {
            "var_smoothing": [0.1, 0.5, 1.0, 1.5, 2.0]
        }
    },
    "KNN": {
        "model": KNeighborsClassifier(),
        "params": {
            "n_neighbors": [3, 5, 10, 20],
            "weights": ["uniform", "distance"]
        }
    }
}
# تشغيل GridSearchCV على كل الموديلات
results = {}
for name, mp in models_params.items():
    grid = GridSearchCV(mp["model"], mp["params"], cv=5, scoring="accuracy", n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = {
        "best_params": grid.best_params_,
        "accuracy": report["accuracy"],
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1_score": report["weighted avg"]["f1-score"]
    }

# طباعة النتائج
for model, metrics in results.items():
    print(f"\nModel: {model}")
    print("Best Params:", metrics["best_params"])
    print("Accuracy:", round(metrics["accuracy"], 4))
    print("Precision:", round(metrics["precision"], 4))
    print("Recall:", round(metrics["recall"], 4))
    print("F1 Score:", round(metrics["f1_score"], 4))


Model: SVM
Best Params: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 0.8708
Precision: 0.8721
Recall: 0.8708
F1 Score: 0.8636

Model: Decision Tree
Best Params: {'max_depth': 3, 'min_samples_split': 2}
Accuracy: 0.8708
Precision: 0.8744
Recall: 0.8708
F1 Score: 0.8625

Model: Naive Bayes
Best Params: {'var_smoothing': 0.5}
Accuracy: 0.8038
Precision: 0.7957
Recall: 0.8038
F1 Score: 0.7913

Model: KNN
Best Params: {'n_neighbors': 20, 'weights': 'uniform'}
Accuracy: 0.7368
Precision: 0.7206
Recall: 0.7368
F1 Score: 0.6752
